### Установка библиотек


In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sqlalchemy import (
    create_engine, 
    MetaData, 
    UniqueConstraint, 
    Table, 
    Column, 
    Integer, 
    String, 
    Float, 
    DateTime, 
    Boolean, 
    ForeignKey,
    Index,
    inspect)
from dotenv import load_dotenv
from datetime import datetime as dt
import os
import io

plt.style.use('ggplot')


In [2]:
ASSETS_DIR = 'assets'

os.makedirs(ASSETS_DIR, exist_ok=True)


In [3]:
# credentials

load_dotenv()

DB_DESTINATION_HOST = os.getenv('DB_DESTINATION_HOST')
DB_DESTINATION_PORT = os.getenv('DB_DESTINATION_PORT')
DB_DESTINATION_USER = os.getenv('DB_DESTINATION_USER')
DB_DESTINATION_PASSWORD = os.getenv('DB_DESTINATION_PASSWORD')
DB_DESTINATION_NAME = os.getenv('DB_DESTINATION_NAME')

### Загрузка данных


In [3]:
data = pd.read_csv('bank_products.csv')


/tmp/ipykernel_3820/1861322736.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('bank_products.csv')


In [4]:
data.head()


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [5]:
data.dtypes


fecha_dato                object
ncodpers                   int64
ind_empleado              object
pais_residencia           object
sexo                      object
age                       object
fecha_alta                object
ind_nuevo                float64
antiguedad                object
indrel                   float64
ult_fec_cli_1t            object
indrel_1mes               object
tiprel_1mes               object
indresi                   object
indext                    object
conyuemp                  object
canal_entrada             object
indfall                   object
tipodom                  float64
cod_prov                 float64
nomprov                   object
ind_actividad_cliente    float64
renta                    float64
segmento                  object
ind_ahor_fin_ult1          int64
ind_aval_fin_ult1          int64
ind_cco_fin_ult1           int64
ind_cder_fin_ult1          int64
ind_cno_fin_ult1           int64
ind_ctju_fin_ult1          int64
ind_ctma_f

## Сохранение данных в БД


In [ ]:
col_types = {}
for el in data.dtypes.items():
    if el[1] == 'object':
        col_types[el[0]] = String
    elif el[1] == 'float64':
        col_types[el[0]] = Float
    elif el[1] == 'int64':
        col_types[el[0]] = Integer


In [ ]:
data['fecha_dato'] = pd.to_datetime(data['fecha_dato'])

In [ ]:
engine = create_engine(f'postgresql://{DB_DESTINATION_USER}:{DB_DESTINATION_PASSWORD}@{DB_DESTINATION_HOST}:{DB_DESTINATION_PORT}/{DB_DESTINATION_NAME}')

metadata = MetaData()
bank_products = Table(
    'bank_products',
    metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    *[Column(el[0], el[1]) for el in col_types.items() if el[0] != 'fecha_dato'],
    Column('fecha_dato', DateTime)
)
Index('idx_fecha_dato', bank_products.c.fecha_dato)

# ИСПРАВЛЕННЫЙ КОД: используем engine вместо conn для создания таблицы
with engine.connect() as conn:
    if not inspect(engine).has_table(bank_products.name): 
        metadata.create_all(engine)  # Используем engine для автоматического коммита
        print(f"Таблица {bank_products.name} успешно создана")
    else:
        print(f"Таблица {bank_products.name} уже существует")


In [ ]:
# Проверка существования таблицы
with engine.connect() as conn:
    if not inspect(engine).has_table(bank_products.name): 
        print('Table does not exist')
    else:
        print('Table exists')


In [ ]:
data.shape


In [ ]:
csv_buffer = io.StringIO()
data.to_csv(csv_buffer, index=False, header=True)
csv_buffer.seek(0)  # Перемотка в начало для чтения

conn = engine.raw_connection()
try:
    cur = conn.cursor()
    try:
        # with open(tmp_path, "r", encoding="utf-8") as f:
        
        cur.copy_expert(f"COPY {bank_products.name} ({', '.join(data.columns)}) FROM STDIN WITH CSV HEADER", csv_buffer)
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"Ошибка при добавлении транзакций в базу данных через COPY: {str(e)}")
        raise
    finally:
        cur.close()
finally:
    conn.close()


## Preprocessing


### Обработка пропусков


In [4]:
engine = create_engine(f'postgresql://{DB_DESTINATION_USER}:{DB_DESTINATION_PASSWORD}@{DB_DESTINATION_HOST}:{DB_DESTINATION_PORT}/{DB_DESTINATION_NAME}')


In [21]:
data = []


with engine.connect() as conn:
    min_date = pd.read_sql_query('select min(fecha_dato) from bank_products', conn)
    max_date = pd.read_sql_query('select max(fecha_dato) from bank_products', conn)
dates = pd.date_range(start=min_date.values[0][0], end=max_date.values[0][0], periods=5)

with engine.connect() as conn:
    for i in range(1,len(dates)):
        date_start = dates[i-1].strftime('%Y-%m-%d')
        date_end = dates[i].strftime('%Y-%m-%d')
        data_part = pd.read_sql_query(f"select * from bank_products where fecha_dato between '{date_start}' and '{date_end}'", conn)
        data.append(data_part)
        print(f'{i} of {len(dates)}')

data = pd.concat(data)


1 of 5
2 of 5
3 of 5
4 of 5


In [23]:
del data_part

In [24]:
data.isna().sum() / data.shape[0]


id                       0.000000
ncodpers                 0.000000
ind_empleado             0.001911
pais_residencia          0.001911
sexo                     0.001916
age                      0.000000
fecha_alta               0.001911
ind_nuevo                0.001911
antiguedad               0.000000
indrel                   0.001911
ult_fec_cli_1t           0.998172
indrel_1mes              0.011877
tiprel_1mes              0.011877
indresi                  0.001911
indext                   0.001911
conyuemp                 0.999867
canal_entrada            0.014585
indfall                  0.001911
tipodom                  0.001911
cod_prov                 0.006723
nomprov                  0.006723
ind_actividad_cliente    0.001911
renta                    0.203297
segmento                 0.014829
ind_ahor_fin_ult1        0.000000
ind_aval_fin_ult1        0.000000
ind_cco_fin_ult1         0.000000
ind_cder_fin_ult1        0.000000
ind_cno_fin_ult1         0.000000
ind_ctju_fin_u

Наибольшая доля пропусков у фичей: **ult_fec_cli_1t и conyuemp**  
**ult_fec_cli_1t** - последняя дата, когда клиент был премиальным. Можно трансформировать фичу в количество дней с даты, когда клиент был премиальным, тогда для текущих премиальных клиентов данная фича будет равна нулю.  
**conyuemp** - 1, если клиент супруг(а) работника. Можно заполнить пропуски нулями.


In [ ]:
data['days_from_premium'] = data['ult_fec_cli_1t'].apply(lambda x: dt.today() - pd.to_datetime(x) if not pd.isna(x) else x)


In [ ]:
data[data['days_from_premium'].notna()]['days_from_premium'].describe()


In [ ]:
data['days_from_premium'] = data.apply(lambda x: 0 if x['indrel_1mes'] == 1 else x['days_from_premium'], axis=1)


In [ ]:
data['conyuemp'].fillna(0, inplace=True)


In [ ]:
data.isna().sum()


In [ ]:
data['fecha_dato'] = pd.to_datetime(data['fecha_dato'])


In [ ]:
data.rename(columns={"fecha_dato":"div_data", "ncodpers": "client_id", "ind_empleado": "empl_status", "pais_residencia": "country",
                     "sexo": "sex", "fecha_alta": "first_contract_date", "ind_nuevo": "is_last_6m_contract", "antiguedad": "work_expirience",
                     "indrel": "is_new_client", "ult_fec_cli_1t": "last_premium_date", "indrel_1mes": "client_type",
                     "tiprel_1mes": "activity_type", "indresi": "is_resident", "indext": "not_resident", "conyuemp": "has_bank_spouse",
                     "canal_entrada": "acquisition_channel", "indfall": "is_acc_actual", "tipodom": "adress_type",
                     "cod_prov": "region", "nomprov": "region_name", "ind_actividad_cliente": "is_active", "renta": "income",
                     "segmento": "segment",
                     "ind_ahor_fin_ult1": "acc_savings",
                     "ind_aval_fin_ult1": "acc_garant",
                     "ind_cco_fin_ult1": "acc_current",
                     "ind_cder_fin_ult1": "acc_derivative",
                     "ind_cno_fin_ult1": "acc_salary",
                     "ind_ctju_fin_ult1": "acc_child",
                     "ind_ctop_fin_ult1": "acc_spec1",        
                     "ind_ctma_fin_ult1": "acc_spec3",
                     "ind_ctpp_fin_ult1": "acc_spec2",
                     "ind_deco_fin_ult1": "acc_short_deposit",
                    "ind_deme_fin_ult1": "acc_middle_deposit",
                    "ind_dela_fin_ult1": "acc_long_deposit",
                    "ind_ecue_fin_ult1": "acc_digital",
                    "ind_fond_fin_ult1": "acc_cash",
                    "ind_hip_fin_ult1": "acc_mortgage",
                    "ind_plan_fin_ult1": "acc_pension",
                    "ind_pres_fin_ult1": "acc_credit",
                    "ind_reca_fin_ult1": "acc_tax",
                    "ind_tjcr_fin_ult1": "acc_credit_cart",
                    "ind_valo_fin_ult1": "acc_securities",
                    "ind_viv_fin_ult1": "acc_home",
                    "ind_nomina_ult1": "acc_salary_payment",
                    "ind_nom_pens_ult1": "acc_pension_loans",
                    "ind_recibo_ult1": "acc_debit"}, inplace=True)


In [ ]:
data['income'].fillna(np.nanmedian(data['income']), inplace=True)


In [ ]:
data[data['sex'].isna()].isna().sum()


Клиенты с незаполненным атрибутом sex имеют много пропусков, такие данные можно удалить


In [ ]:
data = data[data['sex'].notna()]


In [ ]:
data['activity_type'].fillna('P', inplace=True)


In [ ]:
data['acquisition_channel'].fillna('other', inplace=True)


In [ ]:
data['region'].fillna(0, inplace=True)
data['region_name'].fillna('NO_DATA', inplace=True)
data['segment'].fillna('00 - NO SEGMENT', inplace=True)
data['acc_salary_payment'] = data['acc_salary_payment'].fillna(0).astype(int)
data['acc_pension_loans'] = data['acc_pension_loans'].fillna(0).astype(int)


In [ ]:
data['work_expirience'].replace(-999999, 0, inplace=True)
data['is_new_client'].replace(99, 0, inplace=True)


### Определение типов данных


In [ ]:
data['age'] = data['age'].astype('int')
data['work_expirience'] = data['work_expirience'].astype('int')
data['client_type'] = data['client_type'].astype(str)
data['has_bank_spouse'] = data['has_bank_spouse'].astype(str)


In [ ]:
# представляем days_from_premium в вид int
data['days_from_premium'] = data['days_from_premium'].apply(lambda x: x.days if not pd.isna(x) and x != 0 else x)

data['days_from_first_contract'] = dt.today() - pd.to_datetime(data['first_contract_date'])
data['days_from_first_contract'] = data['days_from_first_contract'].apply(lambda x: x.days)


In [ ]:
data['has_bank_spouse'] = data['has_bank_spouse'].map({'S': 1, 'N': 0}).fillna(0)
data['is_acc_actual'] = data['is_acc_actual'].map({'S': 1, 'N': 0}).fillna(0)
data['is_resident'] = data['is_resident'].map({'S': 1, 'N': 0}).fillna(0)
data['is_male'] = data['sex'].map({'V': 1, 'H': 0}).astype(int)


In [ ]:
data['has_bank_spouse'] = data['has_bank_spouse'].astype(int)
data['is_resident'] = data['is_resident'].astype(int)
data['is_acc_actual'] = data['is_acc_actual'].astype(int)
data['is_last_6m_contract'] = data['is_last_6m_contract'].astype(int)
data['is_new_client'] = data['is_new_client'].astype(int)
data['is_active'] = data['is_active'].astype(int)


In [ ]:
data.drop(columns=['last_premium_date', 'first_contract_date', 'sex', 'not_resident', 'adress_type', 'region_name'], inplace=True)


In [ ]:
data.to_parquet('bank_products_processed.parquet')


## EDA


In [ ]:
# запускаем, если нужно
data = pd.read_parquet('bank_products_processed.parquet')


In [ ]:
id_col = ['client_id']
num_features = ['age', 'work_expirience', 'income', 'days_from_first_contract', 'days_from_premium']
object_features = [el for el in data.select_dtypes(['object']).columns.tolist() if el not in num_features + id_col] + ['region']
cat_features = [col for col in data.select_dtypes(['int']).columns.tolist() if col not in num_features + object_features + id_col]


### Описательная статистика


In [ ]:
for el in object_features:
    print(data[el].value_counts(), '\n')


In [ ]:
data[num_features].describe()


In [ ]:
data[cat_features].sum() / data[cat_features].count()


In [ ]:
target_cols = [col for col in data.columns if col.startswith('acc_')]


Выводы по разделу:

1. Больше половины клиентов имеют текущие счета.
2. Только ~46% пользователей - активные.
3. Клиента банка в подавляющем большинстве из Испании.
4. Средний возраст клиентов - около 40 лет (медиана - 39, ср. арфим. - 40).
5. Среднее количество дней с подписания первого договора большое и в среднем.
6. Большинство клиентов имеют премиальный статус


### Графики


In [ ]:
f,ax = plt.subplots(figsize=(15,20))
sns.heatmap(data[cat_features + num_features].corr().loc[[col for col in cat_features + num_features if col not in target_cols], target_cols].T, annot=True, cmap='coolwarm', ax=ax)
plt.title('Корреляция между признаками и целевыми переменными')
plt.savefig(os.path.join(ASSETS_DIR, 'correlation_cat_num_features.png'))
plt.show()


В корреляционной матрице видно, что признаки is_active, age и work_experienc, days_from_first_contract оказывают наиболее значимое влияние на наличие большинства счетов


In [ ]:
f,ax = plt.subplots(figsize=(20,20))
sns.heatmap(data[target_cols].corr(), annot=True, cmap='coolwarm', ax=ax)
plt.title('Корреляция между различными счетами')
plt.savefig(os.path.join(ASSETS_DIR, 'correlation_target_accounts.png'))
plt.show()


In [ ]:
data[target_cols].sum(axis=1).describe()


Наиболее значимая корреляция прослеживается:  
1) между acc_salary_payment и acc_pension_loans, это связано с тем, что клиент с наличием зарплатных обязательств скорее всего имеет и пенсионные обязательства перед работниками, поэтому заводит второй счет;
2) аналогичная логика справедлива и для связки acc_salary с признаками acc_salary_payment, acc_pension_loans;
3) отрицательная корреляция данных показателей с acc_current связана с тем, что клиенты, использующие продукты банка для получения зарплаты, реже заводят дополнительно текущий счет, т.к. все средства находятся на зарплатном счете.


In [ ]:
num_features


In [ ]:
f, axes = plt.subplots(1, 4, figsize=(60, 40))

sns.boxplot(data=data, x='is_active', y='age', ax=axes[0])
sns.boxplot(data=data, x='is_active', y='work_expirience', ax=axes[1])
sns.boxplot(data=data, x='is_active', y='income', ax=axes[2])
sns.boxplot(data=data, x='is_active', y='days_from_first_contract', ax=axes[3])

for ax in axes:
    ax.set_xlabel(ax.get_xlabel(), fontsize=40)
    ax.set_ylabel(ax.get_ylabel(), fontsize=40)
    ax.tick_params(axis='both', which='major', labelsize=40)

plt.tight_layout()
plt.savefig(os.path.join(ASSETS_DIR, 'boxplot_is_active.png'))
plt.show()


### Генерация признаков


In [ ]:
south_regions = [10, 45, 16, 46, 7, 12, 6, 13, 2, 3, 21, 41, 14, 23, 18, 30, 4, 11, 29, 52, 38, 35]


In [ ]:
data['is_capital'] = data['region'].apply(lambda x: 1 if x == 28 else 0)
data['is_barcelona'] = data['region'].apply(lambda x: 1 if x == 8 else 0)
data['is_south'] = data['region'].apply(lambda x: 1 if x in south_regions else 0)


In [ ]:
data['acquisition_channel'].value_counts(normalize=True)


In [ ]:
data['is_KHE'] = data['acquisition_channel'].apply(lambda x: 1 if x == 'KHE' else 0)
data['is_KAT'] = data['acquisition_channel'].apply(lambda x: 1 if x == 'KAT' else 0)
data['is_KFC'] = data['acquisition_channel'].apply(lambda x: 1 if x == 'KFC' else 0)


In [ ]:
data['empl_status'] = data['empl_status'].apply(lambda x: 1 if x in ['A', 'F'] else 0)
data['vip_status'] = data['segment'].apply(lambda x: 1 if x == '01 - TOP' else 0)
data['potential_client'] = data['activity_type'].apply(lambda x: 1 if x == 'R' else 0)


In [ ]:
data['segment'].value_counts()


In [ ]:
object_features = ['empl_status', 'vip_status', 'potential_client', 'is_KHE', 'is_KAT', 'is_KFC', 'is_capital',
                   'is_barcelona', 'is_south']


In [ ]:
f,ax = plt.subplots(figsize=(15,20))
sns.heatmap(data[object_features + target_cols].corr().loc[[col for col in object_features if col not in target_cols], target_cols].T, annot=True, cmap='coolwarm', ax=ax)
plt.title('Корреляция между признаками и целевыми переменными')
plt.savefig(os.path.join(ASSETS_DIR, 'correlation_object_features.png'))
plt.show()


In [ ]:
data['acquisition_channel'].unique()


Выводы по разделу:

1. Наличие ВИП статуса оказывает сильное влияние на факт наличия большинства банковских продуктов
2. Привлечение клиентов через KHE (через внешних партнеров) и KAT (телефонные звонки) также влияют на покупку тех или иных продуктов. Так, привлечение через KHE коррелирует с наличием текущего счета, т.е. клиенты, пришедщие через внешних партнеров банка замотивированы на заведение текущего счета. Привлечение через телефонные звонки чаще мотивирует людей заводить в банке особый счет первого типа. Без понимания особенностей особого счета сложно оценить взаимосвязь.
3. Наличие связи между is_capital и acc_current свидетельствует о том, что клиенты Мадрида реже заводят текущий счет в банке, чаще отдается предпочтение специализированным счетам.


## Логирование в MLFlow


In [ ]:
import mlflow


In [ ]:
# константы для логирования в mlflow

EXPERIMENT_NAME = "final_project_bank_alexdem"


os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5020

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")


In [ ]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id


In [ ]:
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
RUN_NAME = "eda"

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_artifact("EDA.ipynb")
    mlflow.log_artifacts(ASSETS_DIR)


## Сохранение результатов


In [ ]:
data.columns


In [ ]:
cat_features = [col for col in cat_features if col not in target_cols]


In [ ]:
data[num_features + cat_features + target_cols + object_features + id_col + ['div_data']].to_parquet("bank_products_processed.parquet")
